# Stablecoin Simulation

---

This notebook is part of a stablecoin project.

---

# Table of Contents

1. [System Requirements](#1.-System-Requirements)
  * [Requirements Analysis](#Requirements-Analysis)
  * [Visual System Mappings](#Visual-System-Mappings)
  * [Mathematical Specification](#Mathematical-Specification)


2. [System Design](#2.-System-Design)
  * [Differential Specification](#Differential-Specification)
  * [Modelling](#Modelling)
  * [Simulation](#Simulation)


---

# 1. System Requirements

## Requirements Analysis

**Goal:** Simulate a simple collateral-backed stablecoin and examine the effects of external price shocks

**Scope:** Showcasing how simulations can offer insights into specific dynamics

**Question:** How do price shocks affect vault liquidations?

**Assumptions:**
- The price of the underlying collateral asset follows a simple stochastic process
- The _Vault Owners_ maintain their vault positions following a certain strategy
- Liquidations are triggered immediately and the collateral flows into the so called _Residual Ecosystem_

## Visual System Mappings

#### Entity Relationship Diagram

TODO

#### Stock & Flow Diagram

TODO

## Mathematical Specification

TODO

---

# 2. System Design

## Differential Specification

TODO

## Modelling

In [1]:
##############
# 0. IMPORTS #
##############

# Standard libraries
import math
from collections import Counter
import numpy as np
import copy

# model
from model.state_variables import set_initial_state

# utils
import sys
sys.path.append('./utils')
from owner import Owner, OwnerStrategy
from metrics import Metrics
from price_simulator import PriceSimulator
from vault import Vault

# Analysis and plotting modules
import pandas as pd
import plotly.express as px

# radCAD modules
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend


In [2]:
#############
# 0b. UTILS #
#############

INITIAL_COLL_PRICE = 3000
NUM_OWNERS = 100
MIN_COLL_RATIO = 1.5
LIQUIDATION_RATIO = 1/MIN_COLL_RATIO

owner = Owner(OwnerStrategy.TRADITIONAL)
coll_price_simulator = PriceSimulator()
metrics = Metrics(INITIAL_COLL_PRICE)


In [3]:
######################
# 1. STATE VARIABLES #
######################

initial_state = set_initial_state(INITIAL_COLL_PRICE, NUM_OWNERS, LIQUIDATION_RATIO)
initial_state

{'colleteral': Collateral(collateral_price=3000),
 'owners': [Owner(vault=Vault(collateral_balance=291.38219656410877, debt_balance=128070.35454996745, blocked=False), wallet=Wallet(collateral_balance=9.096464035962471, stable_coin_balance=5066.022371025668), strategy=<OwnerStrategy.TRADITIONAL: 6>),
  Owner(vault=Vault(collateral_balance=201.266229113879, debt_balance=52960.26786815389, blocked=False), wallet=Wallet(collateral_balance=7.374853499846513, stable_coin_balance=7338.869502145053), strategy=<OwnerStrategy.RANDOM: 3>),
  Owner(vault=Vault(collateral_balance=324.21268760800206, debt_balance=39443.79100046427, blocked=False), wallet=Wallet(collateral_balance=9.348221193930424, stable_coin_balance=13955.279605215477), strategy=<OwnerStrategy.SIMPLE_COLL: 4>),
  Owner(vault=Vault(collateral_balance=236.77477448904773, debt_balance=30947.190294654232, blocked=False), wallet=Wallet(collateral_balance=6.149004173634701, stable_coin_balance=2586.5461287696276), strategy=<OwnerStrate

In [4]:
######################
# 1. STATE VARIABLES #
######################

initial_state = {
    'coll_price_change_rate': None,
    'coll_price': INITIAL_COLL_PRICE,
    'negative_shocks': 0,
    'vault': Vault(INITIAL_COLL_PRICE, 2000, 1.5, 1000000)
}


########################
# 2. SYSTEM PARAMETERS #
########################

system_params = {}


#######################
# 3. POLICY FUNCTIONS #
#######################

def p_coll_price_change(params, substep, state_history, previous_state):
    coll_price_change_rate, negative_shock_occurence = coll_price_simulator.calculate_change_rate()
    return {
        'coll_price_change_rate': coll_price_change_rate,
        'negative_shock_occurence': negative_shock_occurence
    }


#############################
# 4. STATE UPDATE FUNCTIONS #
#############################

def s_coll_price_change_rate(params, substep, state_history, previous_state, policy_input):
    coll_price_change_rate = policy_input['coll_price_change_rate']
    return 'coll_price_change_rate', coll_price_change_rate

def s_coll_price(params, substep, state_history, previous_state, policy_input):
    coll_price = previous_state['coll_price'] * (1 + policy_input['coll_price_change_rate'])
    return 'coll_price', coll_price

def s_negative_shocks(params, substep, state_history, previous_state, policy_input):
    negative_shocks = previous_state['negative_shocks']
    if policy_input['negative_shock_occurence']:
        negative_shocks += 1
    return 'negative_shocks', negative_shocks

def s_vault(params, substep, state_history, previous_state, policy_input):
    vault = copy.deepcopy(previous_state['vault'])
    vault.update_coll_price(previous_state['coll_price'])
    metrics.update_indicators(previous_state['coll_price'])
    owner.modify_vault(vault, metrics.get_indicators())
    return 'vault', vault


##################################
# 5. PARTIAL STATE UPDATE BLOCKS #
##################################

state_update_blocks = [
    {
        'policies': {
            'coll_price_change': p_coll_price_change
        },
        'variables': {
            'coll_price_change_rate': s_coll_price_change_rate,
            'coll_price': s_coll_price,
            'negative_shocks': s_negative_shocks
        }
    },
    {
        'policies': {},
        'variables': {
            'vault': s_vault
        }
    }
]

## Simulation

In [5]:
####################
# 6. CONFIGURATION #
####################

model = Model(
    initial_state=initial_state,
    state_update_blocks=state_update_blocks,
    # params=system_params
)

simulation = Simulation(
    model=model,
    timesteps=300,  # Number of timesteps
    runs=10  # Number of Monte Carlo Runs
)

experiment = Experiment([simulation])
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS, drop_substeps=True)


################
# 7. EXECUTION #
################

raw_result = experiment.run()


####################################
# 8. SIMULATION OUTPUT PREPARATION #
####################################

simulation_result = pd.DataFrame(raw_result)

simulation_result['coll_balance'] = [t.coll_balance for t in simulation_result['vault']]
simulation_result['coll_value'] = [t.coll_value for t in simulation_result['vault']]
simulation_result['loan_value'] = [t.loan_value for t in simulation_result['vault']]
simulation_result['liquidated'] = [t.liquidated for t in simulation_result['vault']]
simulation_result['liquidated_value'] = [t.liquidated_value for t in simulation_result['vault']]
simulation_result['liquidation_buffer'] = [t.liquidation_buffer for t in simulation_result['vault']]
simulation_result['volatility_buffer'] = [t.volatility_buffer for t in simulation_result['vault']]

simulation_result.drop(columns=['vault'], inplace=True)

simulation_result.head(10)


,coll_price_change_rate,coll_price,negative_shocks,simulation,subset,run,substep,timestep,coll_balance,coll_value,loan_value,liquidated,liquidated_value,liquidation_buffer,volatility_buffer
0,NaN,3000.000000,0,0,0,1,0,0,2000.000000,6.000000e+06,1000000,False,0.0,3.000000e+06,2.000000e+06
1,-0.016190,2951.431234,0,0,0,1,2,1,1016.456005,3.000000e+06,1000000,False,0.0,1.000000e+06,1.000000e+06
2,0.030088,3040.232856,0,0,0,1,2,2,1016.456005,3.090263e+06,1000000,False,0.0,1.060175e+06,1.030088e+06
3,-0.000693,3038.126865,0,0,0,1,2,3,1016.456005,3.088122e+06,1000000,False,0.0,1.058748e+06,1.029374e+06
4,-0.087628,2771.902206,0,0,0,1,2,4,1016.456005,2.817517e+06,1000000,False,0.0,8.783444e+05,9.391722e+05
5,-0.053270,2624.241664,0,0,0,1,2,5,1016.456005,2.667426e+06,1000000,False,0.0,7.782841e+05,8.891421e+05
6,0.051940,2760.545030,0,0,0,1,2,6,1016.456005,2.805973e+06,1000000,False,0.0,8.706484e+05,9.353242e+05
7,0.044872,2884.416771,0,0,0,1,2,7,1016.456005,2.931883e+06,1000000,False,0.0,9.545885e+05,9.772942e+05
8,-0.009578,2856.790903,0,0,0,1,2,8,1016.456005,2.903802e+06,1000000,False,0.0,9.358682e+05,9.679341e+05
9,0.021380,2917.868176,0,0,0,1,2,9,1016.456005,2.965885e+06,1000000,False,0.0,9.772564e+05,9.886282e+05


In [6]:
########################################
# 9. SIMULATION ANALYSIS: SHOCK EVENTS #
########################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['negative_shocks'],
    title="Negative Market Shocks",
    labels={"value": "event number"},
    color='run'
)

In [7]:
###################################################
# 9. SIMULATION ANALYSIS: COLLATERAL PRICE CHANGE #
###################################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['coll_price_change_rate'],
    title="Collateral Price Change Rate",
    color='run'
)

In [8]:
############################################
# 9. SIMULATION ANALYSIS: COLLATERAL PRICE #
############################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['coll_price'],
    title="Collateral Price",
    color='run'
)

In [9]:
############################################
# 9. SIMULATION ANALYSIS: COLLATERAL VALUE #
############################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['coll_value'],
    title="Vault",
    color='run'
)

In [10]:
##################################
# 9. SIMULATION ANALYSIS: VAULTS #
##################################

px.bar(
    simulation_result,
    x='run',
    y=['loan_value', 'liquidation_buffer', 'volatility_buffer', 'liquidated_value'],
    title="Vault",
    animation_frame='timestep',
)